In [1]:
import numpy as np
import pandas as pd
import re
import gensim
import os

import warnings
#warnings.filterwarnings('ignore')


/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#!pip install gensim

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from nltk import word_tokenize
from tqdm import tqdm_notebook as tqdm

#  Loading the dataset 

In [4]:
df = pd.read_csv('Preprocessed Fake Reviews Detection Dataset.csv')


In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.dropna(inplace=True)

In [7]:
df['length'] = df['text_'].apply(len)

In [8]:
data_classes = ["CG", "OR"]

df['label'] = df['label'].apply(data_classes.index)


In [9]:
df.head()

,category,rating,label,text_,length
0,Home_and_Kitchen_5,5.0,0,love well made sturdi comfort i love veri pretti,48
1,Home_and_Kitchen_5,5.0,0,love great upgrad origin i 've mine coupl year,46
2,Home_and_Kitchen_5,5.0,0,thi pillow save back i love look feel pillow,44
3,Home_and_Kitchen_5,1.0,0,miss inform use great product price i,37
4,Home_and_Kitchen_5,5.0,0,veri nice set good qualiti we set two month,43


In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from nltk import word_tokenize
from tqdm import tqdm_notebook as tqdm

#  Train-Test Split 

In [11]:
review_train, review_test, label_train, label_test = train_test_split(df['text_'],df['label'],test_size=0.35)

In [12]:
type (review_train)

pandas.core.series.Series

In [13]:
review_train

37303    the bag excel pocket wide tight i need small s...
27745    i love seri one i pre-ord autobuy jenna bennet...
2766     i realli love steam cloth iron it 's sooo much...
24057                 i like just n't find good first book
34720    thi huge pack fun kid love my son love one i b...
                               ...                        
35069    cute mlp plush it 's soft cuddli my daughter l...
22802    we use coupl year i continu use long i the pla...
10406    superb finish seriou photographi tool thi came...
25824    love sarcasm banter blake jennif howev i could...
21160    we older dog need joint they love tast take pr...
Name: text_, Length: 26280, dtype: object

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/u188274/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Doc2Vec Model to build vocabularies

In [15]:
%%time
#The Doc2Vec model takes 'tagged_documents'

tagged_tr = [TaggedDocument(words=word_tokenize(_d.lower()),\
tags=[str(i)]) for i, _d in enumerate(review_train)]

# tagged_val = [TaggedDocument(words=word_tokenize(_d.lower()),\
# tags=[str(i)]) for i, _d in enumerate(val_data.REVIEW_TEXT)]

tagged_test = [TaggedDocument(words=word_tokenize(_d.lower()),\
tags=[str(i)]) for i, _d in enumerate(review_test)]

CPU times: user 5.84 s, sys: 72.2 ms, total: 5.91 s
Wall time: 5.93 s


In [16]:
model = load_model("doc2vec.model")

NameError: name 'load_model' is not defined

In [17]:
%%time
#Instantiate the model
model = Doc2Vec(vector_size=60, 
                window=5, 
                alpha=.025, #initial learning rate
                min_alpha=0.00025, #learning rate drops linearly to this
                min_count=2, #ignores all words with total frequency lower than this.
                dm =1, #algorithm 1=distributed memory
                workers=16)#cores to use

#build the vocab on the training data
model.build_vocab(tagged_tr)

CPU times: user 5.57 s, sys: 32.3 ms, total: 5.6 s
Wall time: 5.6 s


In [18]:
print (model.corpus_count, model.epochs, len(tagged_tr))


26280 5 26280


# Training Doc2VecModel

In [79]:
%%time
max_epochs = 15

for epoch in range(max_epochs):
    model.train(tagged_tr,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

CPU times: user 12min 44s, sys: 2min 29s, total: 15min 14s
Wall time: 8min 37s


In [ ]:
#!pip install ipywidgets
model.save("doc2vec.model")



# Finding semantically similar words using Doc2VecModel 

In [ ]:
model.wv.most_similar("nice")

#  Tokenize the corpus using Tokenizer 

In [ ]:
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

from keras.models import Model,Sequential
from keras.layers import Input,concatenate,Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D,Bidirectional, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review_train)
vocab_size = len(tokenizer.word_index)+1
print('Vocab Size is ',vocab_size)



# ** Building the embeddings using the vocabulary the corpus using Tokenizer **

In [ ]:
embedding_matrix = np.zeros((vocab_size, 60))
for i, row in review_train.items():
    #print (i, row)
    if i >= vocab_size - 1:
        continue
    if (str(i) in model.docvecs):
        embedding = model.docvecs[str(i)]
        if embedding is not None:
            embedding_matrix[i+1] = embedding


#  Converting the tokens to sequences and padding them to sequence length

In [ ]:

%%time 
SEQUENCE_LENGTH = 100
x_data = pad_sequences(tokenizer.texts_to_sequences(review_train) , maxlen = SEQUENCE_LENGTH)
y_data = label_train
y_data = y_data.values.reshape(-1,1)

# vx_data = pad_sequences(tokenizer.texts_to_sequences(val_data.REVIEW_TEXT) , maxlen = SEQUENCE_LENGTH)
# vy_data = val_data.LABEL
# vy_data = vy_data.values.reshape(-1,1)

testx = pad_sequences(tokenizer.texts_to_sequences(review_test) , maxlen = SEQUENCE_LENGTH)
testy = label_test
testy = testy.values.reshape(-1,1)

#  Building the Model using LSTM and OneDNN is enabled 

In [ ]:
nlp_input = Input(shape=(SEQUENCE_LENGTH,)) 
                   
emb=Embedding(vocab_size,60,weights=[embedding_matrix],input_length=SEQUENCE_LENGTH)(nlp_input)
 # layers.Embedding(encoder.vocab_size, embedding_dim),

nlp_out = LSTM(64)(emb)                         
classifier1 = Dense(128, activation='relu')(nlp_out) 
dropout = Dropout(0.2)(classifier1) 
classifier2 = Dense(32, activation='relu')(dropout) 
output = Dense(1, activation='sigmoid')(classifier2) 

model = Model(inputs=[nlp_input], outputs=[output])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
%%time 
history = model.fit(x_data,y_data,batch_size=32,epochs=5)

In [ ]:
results = model.evaluate(testx, testy)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(testx[:3])
print("predictions shape:", predictions.shape)

In [28]:
import pickle

pickle.dump(model, open('model.pkl','wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dropout
.........vars
......embedding
.........vars
............0
......input_layer
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                    